<h3> Defined Stochastic Strategy </h3>

In [8]:
from PerformantStochasticBacktestingModule import StochasticBacktesting
import polars as pl
from polars import col
from itertools import product, combinations
pl.Config.set_tbl_rows(31)
pl.Config.set_tbl_cols(13)

polars.config.Config

short (caindo)
<ul>
    <li>15 mins: entre 100 a 25 ->  (exemplo: dia 22/03/2023 as 15:15 -região 75) (linha vermelha (D) por cima da linha azul (K))</li>
    <li>5 mins: entre 100 a 25 -> (exemplo: dia 22/03/2023 as 15:15 -região 50) (linha vermelha (D) por cima da linha azul (K))</li>
    <li>3 mins: entre 100 a 25->  (exemplo: dia 22/03/2023 as 15:15 -região 50) (linha vermelha (D) por cima da linha azul (K))</li>
    <li>STOP: quando 15 mins reverter linha azul (K) em cima da linha vermelha (D)</li>
</ul>

long (subindo)
<ul>
    <li>15 mins: entre 0 a 33 (exemplo: dia 13/03/2023 as 06:45 -região 15) (linha azul (K) por cima da vermelha (D))</li>
    <li>5 mins: entre 0 a 33 (exemplo:  dia 13/03/2023 as 06:45 - região 11,5)  (linha azul (K) por cima da vermelha (D))</li>
    <li>3 mins: entre 0 a 33 (exemplo:  dia 13/03/2023 as 06:45 - região 22)  (linha azul (K) por cima da vermelha (D))</li>
    <li>STOP: quando 15 mins reverter a linha vermelha (D) em cima da azul (K) (exemplo dia 13/03/2023 ao 12:00)</li>
</ul>


In [9]:
import os.path


class KlapauciusStrategy(StochasticBacktesting):
    def __init__(self, symbols: list, date: str, windows_range: list, limit=720, amount=1000, use_spread=True, log_level=0, avoid_loss_lock=False, adjust_trend=True):
        super().__init__(symbols, date, limit, amount,
                         use_spread, log_level, avoid_loss_lock)
        anchor_symbol, _ = symbols
        self.adjust_trend = adjust_trend
        self.set_multi_stochastic_windows(anchor_symbol, windows_range)

    def test_and_save_date_combinations(self, date: str, windows: list, upper_limit: int, reversal_window_index: int):
        windows_label = "{}-{}-{}".format(windows[0], windows[1], windows[2])
        symbols_data_directory = "{}/{}_{}".format(
            self.parquet_data_directory, self.symbols[0], self.symbols[1])
        filename = "windows={}&upper_limit={}&reversal_window_index={}&avoid_loss_lock={}.parquet".format(
            windows_label, upper_limit, reversal_window_index, 1 if self.avoid_loss_lock else 0)
        file_path = "{}/{}".format(symbols_data_directory, filename)
        file_exists = os.path.isfile(file_path)
        if file_exists:
            return

        performance, buy_and_hold_perf_diff, success_rate, trades, trading_costs_sum, profits_sum, losses_sum, cumulative_returns, mean_returns, risk = self.test_strategy_iteractively(
            date, windows, upper_limit, reversal_window_index
        )

        result_df = pl.DataFrame(
            {
                "date": date,
                "windows": "'{}-{}-{}".format(windows[0], windows[1], windows[2]),
                "upper_limit": upper_limit,
                "reversal_window_index": reversal_window_index,
                "performance": performance,
                "buy_and_hold_perf_diff": buy_and_hold_perf_diff,
                "success_rate": success_rate,
                "trades": trades,
                "trading_costs_sum": trading_costs_sum,
                "profits_sum": profits_sum,
                "losses_sum": losses_sum,
                "cumulative_returns": cumulative_returns,
                "mean_returns": mean_returns,
                "risk": risk,
            },
            schema=self.result_cols_schema
        )

        result_df.write_parquet("{}.parquet".format(file_path))

    def test_strategy(
            self,
            date: str,
            windows: list,
            upper_limit: int,
            reversal_window_index: int
    ):
        anchor_symbol, traded_symbol = self.symbols

        self.set_tests_data(anchor_symbol, windows,
                            upper_limit, reversal_window_index)

        anchor_close = "{}_close".format(anchor_symbol)
        traded_close = "{}_close".format(traded_symbol)

        df_count = self.data.filter(col(anchor_close).is_not_null() & col(
            traded_close).is_not_null()).select(pl.count())[0, 'count']
        position_arr = []
        if self.data.is_empty() or df_count == 1:
            print('FAULTY STRATEGY, windows={}, upper_limit={}, reversal_window_index={}'.format(
                windows, reversal_window_index, reversal_window_index))
            return
        for bar in range(df_count - 1):
            if (self.data[bar, "short"] == 1) & (self.position == 0):
                self.position = -1
            elif (self.data[bar, "long"] == 1) & (self.position == 0):
                self.position = 1
            elif (self.data[bar, "short_close"] == 1) & (self.position == -1):
                self.position = 0
            elif (self.data[bar, "long_close"] == 1) & (self.position == 1):
                self.position = 0
            position_arr.append(self.position)
        position_arr.append(0)
        self.data = self.data.filter(col(anchor_close).is_not_null() & col(traded_close).is_not_null()).with_columns(
            pl.lit(position_arr).alias("position")
        )

        closing_bar = bar + 1
        return self.set_performance()

    def test_strategy_iteractively(
        self,
        date: str,
        windows: list,  # OPTIMIZATION POINT
        upper_limit: int,
        reversal_window_index: int,
        leverage: int = 1
    ):
        anchor_symbol, traded_symbol = self.symbols
        # printout
        if self.log_level > 0:
            stm = "{} | Testing Klapaucius Strategy Iteractively | {}".format(
                self.date, traded_symbol)
            print(100 * "-")
            print(stm)
            print(100 * "-")

        self.set_tests_data(anchor_symbol, windows,
                            upper_limit, reversal_window_index)

        anchor_close = "{}_close".format(anchor_symbol)
        traded_close = "{}_close".format(traded_symbol)

        df_count = self.data.filter(col(anchor_close).is_not_null() & col(
            traded_close).is_not_null()).select(pl.count())[0, 'count']
        position_arr = []
        if self.data.is_empty() or df_count == 1:
            print('FAULTY STRATEGY, windows={}, upper_limit={}, reversal_window_index={}'.format(
                windows, reversal_window_index, reversal_window_index))
            return 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
        for bar in range(df_count - 1):
            if self.position == 0:
                if (self.data[bar, "short"] == 1):
                    if self.go_short(anchor_symbol, traded_symbol, bar, amount="all"):
                        self.position = -1
                elif (self.data[bar, "long"] == 1):
                    if self.go_long(anchor_symbol, traded_symbol, bar, amount="all"):
                        self.position = 1
            else:
                if (self.data[bar, "short_close"] == 1) & (self.position == -1):
                    if self.close_short(anchor_symbol, traded_symbol, bar):
                        self.position = 0
                elif (self.data[bar, "long_close"] == 1) & (self.position == 1):
                    if self.close_long(anchor_symbol, traded_symbol, bar):
                        self.position = 0
            position_arr.append(self.position)
        position_arr.append(0)
        self.data = self.data.filter(col(anchor_close).is_not_null() & col(traded_close).is_not_null()).with_columns(
            pl.lit(position_arr).alias("position")
        )

        closing_bar = bar + 1
        self.close_current_position(anchor_symbol, traded_symbol, closing_bar)
        return self.set_iteractive_performance(
            date, traded_symbol, "Klapaucius Strategy"
        )

    def set_tests_data(
        self,
        anchor_symbol: str,
        windows: list,
        upper_threshold: int,  # 75
        close_window_index: int
    ):

        # reset
        self.clear_transactions()

        trend_col = "{}_trend".format(anchor_symbol)
        short_threshold_col = "short_threshold"
        long_threshold_col = "long_window"

        # upper_limit = 70
        lower_threshold = 100 - upper_threshold  # 25

        self.data = self.data.with_columns(
            pl.when((col(trend_col) == "bullish") & self.adjust_trend).then(
                upper_threshold).otherwise(lower_threshold).alias(short_threshold_col)
        )

        self.data = self.data.with_columns(
            pl.when((col(trend_col) == "bullish") & self.adjust_trend).then(
                lower_threshold).otherwise(upper_threshold).alias(long_threshold_col)
        )

        # short_windows = []
        # long_windows = []

        for i, window in enumerate(windows):
            stochastic_k_col = "{}_K%_{}".format(anchor_symbol, window)
            stochastic_d_col = "{}_D%_{}".format(anchor_symbol, window)

            windowed_short_threshold_col = "short_{}".format(i+1)
            # short_windows.append(short_window)
            windowed_long_threshold_col = "long_{}".format(i+1)
            # long_windows.append(long_window)

            self.data = self.data.with_columns(
                pl.when(
                    (col(stochastic_k_col) > col(short_threshold_col)) &
                    (col(stochastic_d_col) > col(stochastic_k_col))
                ).then(1).otherwise(0).alias(windowed_short_threshold_col)
            )

            self.data = self.data.with_columns(
                pl.when(
                    (col(stochastic_k_col) < col(long_threshold_col)) &
                    (col(stochastic_d_col) < col(stochastic_k_col))
                ).then(1).otherwise(0).alias(windowed_long_threshold_col)
            )

        closing_window = windows[close_window_index]

        short_close_k_col = "{}_K%_{}".format(
            anchor_symbol, closing_window)
        short_close_d_col = "{}_D%_{}".format(
            anchor_symbol, closing_window)
        # review values
        self.data = self.data.with_columns(
            pl.when(
                (col(short_close_k_col) > lower_threshold) &
                (col(short_close_k_col) > col(short_close_d_col)) &
                (col(short_close_k_col).shift(1) <
                 col(short_close_d_col).shift(1))
            ).then(1).otherwise(0).alias("short_close")
        )

        # review values
        long_close_k_col = "{}_K%_{}".format(
            anchor_symbol, closing_window)
        long_close_d_col = "{}_D%_{}".format(
            anchor_symbol, closing_window)

        self.data = self.data.with_columns(
            pl.when(
                (col(long_close_k_col) < upper_threshold) &
                (col(long_close_k_col) < col(long_close_d_col)) &
                (col(long_close_k_col).shift(1) >
                 col(long_close_d_col).shift(1))
            ).then(1).otherwise(0).alias("long_close")
        )

        self.data = self.data.with_columns(
            pl.when(
                (col("long_1") == 1) &
                (col("long_2") == 1) &
                (col("long_3") == 1)
            ).then(1).otherwise(0).alias("long")
        )

        self.data = self.data.with_columns(
            pl.when(
                (col("short_1") == 1) &
                (col("short_2") == 1) &
                (col("short_3") == 1)
            ).then(1).otherwise(0).alias("short")
        )


In [10]:
current_balance = 1000
# current_profit_percent = 1
# # windows = [15, 5, 3]

symbols = ["BTCUSDT", "AGIXUSDT"]
# traded_symbol = symbols[1]



# window_take = 1
# window_max = 20
# windows_range = [window_max]
# limit_take = 10
# i_range = 3

# for w in range(0, 20, window_take):
#     if w == 0:
#         continue
#     windows_range.append(w)
# windows_comb = combinations(windows_range, 3)
# limit_ranges = []
# for l in range(0, 90, limit_take):
#     if l == 0:
#         continue
#     limit_ranges.append(l)

# index_range = []
# for i in range(0, i_range):
#     index_range.append(i)

# combinations = list(product(windows_comb, limit_ranges, index_range))

# cols_schema = [
#     ("date", pl.Utf8),
#     ("windows", pl.Utf8),
#     ("upper_limit", pl.Int32),
#     ("reversal_window_index", pl.Int32),
#     ("performance", pl.Float64),
#     ("buy_and_hold_perf_diff", pl.Float64),
#     ("success_rate", pl.Float64),
#     ("trades", pl.Float64),
#     ("trading_costs_sum", pl.Float64),
#     ("profits_sum", pl.Float64),
#     ("losses_sum", pl.Float64),
#     ("cumulative_returns", pl.Float64),
#     ("mean_returns", pl.Float64),
#     ("risk", pl.Float64),
# ]

# performance_df = pl.DataFrame(
#     schema=cols_schema
# )

test_range = (23, 24)
combinations = [((1, 3, 5), 70, 2)] # accepts reversal
windows_range = [1, 3, 5] # windows_range = [5, 9, 12]
# # combinations = [((6, 11, 12), 60, 2)] # doesnt accept reversal
# # windows_range = [6, 11, 12]
# combinations_length = len(combinations)
# print("COMBINATIONS LENGTH:", combinations_length)


for index in range(test_range[0], test_range[1]):
    day = index + 1
    date = "{}-05-2023".format(day)
    ks = KlapauciusStrategy(
        symbols,  # AGIXUSDT
        date=date,
        windows_range=windows_range,
        amount=current_balance,
        log_level=0,
        avoid_loss_lock=False,
        use_spread=True
    )
    for comb in combinations:
        windows, upper_limit, reversal_window_index = comb
        windows = tuple(sorted(windows))
        # performance, buy_and_hold_perf_diff, success_rate, trades, trading_costs_sum, profits_sum, losses_sum, cumulative_returns, mean_returns, risk = ks.test_strategy(date, windows, upper_limit, reversal_window_index)
        performance, buy_and_hold_perf_diff, success_rate, trades, trading_costs_sum, profits_sum, losses_sum, cumulative_returns, mean_returns, risk = ks.test_strategy_iteractively(date, windows, upper_limit, reversal_window_index)
        result = pl.DataFrame(
            {
                "date": date,
                "windows": "'{}-{}-{}".format(windows[0], windows[1], windows[2]),
                "upper_limit": upper_limit,
                "reversal_window_index": reversal_window_index,
                "performance": performance,
                "buy_and_hold_perf_diff": buy_and_hold_perf_diff,
                "success_rate": success_rate,
                "trades": trades,
                "trading_costs_sum": trading_costs_sum,
                "profits_sum": profits_sum,
                "losses_sum": losses_sum,
                "cumulative_returns": cumulative_returns,
                "mean_returns": mean_returns,
                "risk": risk,
            },
            schema=ks.result_cols_schema
        )
        # current_balance = ks.current_balance
        # performance_df = pl.concat([performance_df, result])
    # filename = "date={}&combinations_l={}&window_take={}&limit_take={}&reversal_i_range={}.csv".format(date, combinations_length, window_take, limit_take, i_range)
    # save_directory = "{}/{}/{}".format(
    #     './tests', traded_symbol, filename)
    # performance_df.write_csv(save_directory)



DATA@@@ shape: (1_299, 5)
┌─────────────────────┬──────────────┬──────────────┬─────────────┬───────────────┐
│ start_time          ┆ BTCUSDT_open ┆ BTCUSDT_high ┆ BTCUSDT_low ┆ BTCUSDT_close │
│ ---                 ┆ ---          ┆ ---          ┆ ---         ┆ ---           │
│ datetime[μs]        ┆ f64          ┆ f64          ┆ f64         ┆ f64           │
╞═════════════════════╪══════════════╪══════════════╪═════════════╪═══════════════╡
│ 2023-05-24 00:00:00 ┆ 27161.75     ┆ 27161.76     ┆ 27161.75    ┆ 27161.76      │
│ 2023-05-24 00:01:00 ┆ 27161.76     ┆ 27161.76     ┆ 27161.75    ┆ 27161.75      │
│ 2023-05-24 00:02:00 ┆ 27161.75     ┆ 27161.76     ┆ 27161.75    ┆ 27161.75      │
│ 2023-05-24 00:03:00 ┆ 27161.75     ┆ 27161.76     ┆ 27161.75    ┆ 27161.76      │
│ 2023-05-24 00:04:00 ┆ 27161.75     ┆ 27164.59     ┆ 27161.75    ┆ 27164.59      │
│ 2023-05-24 00:05:00 ┆ 27164.58     ┆ 27164.59     ┆ 27157.94    ┆ 27157.94      │
│ 2023-05-24 00:06:00 ┆ 27157.95     ┆ 27157.95   

In [11]:
# argmax = performance_df.get_column("performance").arg_max()
# print(argmax)
# print(performance_df.top_k(5, descending=False, by='performance'))
# print(current_balance)

# print(performance_df)

# for index in range(test_range[0], test_range[1]):
    